# figure 1B

In [ ]:
import pandas as pd, numpy as np, mavenn
import src.landscape_tools as lstoo, src.plot_tools as plottoo
import matplotlib as mpl, matplotlib.pyplot as plt, matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable

# LaTeX font for plots
plt.rcParams.update({
    'font.family': 'serif',  # use serif/main font for text elements
    'text.usetex': True,     # use inline math for ticks
})

In [ ]:
# length of sequence and number of spin states
L, q = 10, 2
# number/name of all sites
sites = [26, 27, 28, 31, 35, 50, 53, 56, 57, 58]
# names of key mutations (X=V/L/I)
muts = ['G26E', 'F27X', 'T28I', 'S31R', 'S35T', 'V50L', 'S53P', 'S56T', 'T57A', 'Y58F']

In [ ]:
# mapping site number to vector index
pos2i = {pos: i for i, pos in enumerate(sites)}

# load sequence count data
data = pd.read_csv('data/COV107_mutlib_fit_filtered_exp.tsv', sep='\t')

# convert column 'mut' in the data file to spin chains of 0/1s
seqs = []
for x in range(len(data)):
    seq = [0 for i in range(L)]
    for mut in data['mut'][x].split('-'):
        if mut != 'WT':
            seq[pos2i[int(mut[1:-1])]] = 1
    seqs.append(tuple(seq))
data['mut'] = seqs

# group by sequence and sum sequence counts & rename and drop columns
data.drop(columns=['mutclass','exp1_enrich','exp2_enrich'], inplace=True)
data.rename(columns={'mut':'seq', 'input_Count':'ni', 'exp1_count':'no1', 'exp2_count':'no2'}, inplace=True)
data = data.groupby('seq').sum().reset_index()

# compute empirical enrichments as log-enrichments
data['F1_emp'], data['F2_emp'] = np.log((1.+data.no1) / (1.+data.ni)), np.log((1.+data.no2) / (1.+data.ni))

# enumerate all possible sequences and sort dataframe
seqs = lstoo.seqlist(q=q, L=L)
data['seq'] = pd.Categorical(data['seq'], categories=seqs, ordered=True)
data.sort_values('seq').reset_index(drop=True)

# subtract offset to have germline at zero fitness
data['F1_emp'] -= data.F1_emp.iloc[0]
data['F2_emp'] -= data.F2_emp.iloc[0]

## fit global epistasis model

In [ ]:
# make Pandas dataframe with training/test data (here: all data is training data)
np.random.seed(1)
seqs_str = [''.join([str(a) for a in s]) for s in data.seq]
data_df = pd.DataFrame({'x': seqs_str, 'y': data.F1_emp,
                        'set': [np.random.choice(['training', 'test'], p=[1., 0.]) for s in seqs]})

# separate test from data_df (here: all data is training data)
ix_test = data_df['set'] == 'test'
test_df = data_df[ix_test].reset_index(drop=True)
print(f'number of test sequences: {len(test_df):,}')

# remove test data from data_df
data_df = data_df[~ix_test].reset_index(drop=True)
print(f'sequences to be used for training + validation: {len(data_df):,}')

In [ ]:
# define model (boolean alphabet, additive latent phenotype)
model = mavenn.Model(L=L,
                     alphabet=['0', '1'],
                     gpmap_type='additive',
                     regression_type='GE',
                     ge_noise_model_type='Gaussian',
                     ge_heteroskedasticity_order=0,
                     ge_nonlinearity_hidden_nodes=1)

# set training data
model.set_data(x=data_df.x,
               y=data_df.y,
               validation_frac = .2,
               shuffle=True)

# fit model to data
history = model.fit(learning_rate=.005,
                    epochs=1000,
                    batch_size=q**L,
                    early_stopping=False,
                    linear_initialization=False)

# save model
model.save('output/1b_repl1')

1/1 [==============================] - 0s 56ms/step - loss: 688.8416 - I_var: 0.6177 - val_loss: 166.9734 - val_I_var: 0.5769
Epoch 59/1000
1/1 [==============================] - 0s 50ms/step - loss: 685.8514 - I_var: 0.6229 - val_loss: 166.4172 - val_I_var: 0.5810
Epoch 60/1000
1/1 [==============================] - 0s 57ms/step - loss: 682.9299 - I_var: 0.6279 - val_loss: 165.8388 - val_I_var: 0.5853
Epoch 61/1000
1/1 [==============================] - 0s 42ms/step - loss: 680.0579 - I_var: 0.6329 - val_loss: 165.2299 - val_I_var: 0.5899
Epoch 62/1000
1/1 [==============================] - 0s 52ms/step - loss: 677.2209 - I_var: 0.6379 - val_loss: 164.5988 - val_I_var: 0.5946
Epoch 63/1000
1/1 [==============================] - 0s 42ms/step - loss: 674.4179 - I_var: 0.6427 - val_loss: 163.9631 - val_I_var: 0.5993
Epoch 64/1000
1/1 [==============================] - 0s 57ms/step - loss: 671.6470 - I_var: 0.6476 - val_loss: 163.3378 - val_I_var: 0.6040
Epoch 65/1000
1/1 [===============

1/1 [==============================] - 0s 39ms/step - loss: 558.6496 - I_var: 0.8440 - val_loss: 138.8619 - val_I_var: 0.7860
Epoch 117/1000
1/1 [==============================] - 0s 41ms/step - loss: 557.0135 - I_var: 0.8468 - val_loss: 138.5045 - val_I_var: 0.7886
Epoch 118/1000
1/1 [==============================] - 0s 42ms/step - loss: 555.3934 - I_var: 0.8496 - val_loss: 138.1517 - val_I_var: 0.7913
Epoch 119/1000
1/1 [==============================] - 0s 56ms/step - loss: 553.7897 - I_var: 0.8524 - val_loss: 137.8025 - val_I_var: 0.7939
Epoch 120/1000
1/1 [==============================] - 0s 57ms/step - loss: 552.2028 - I_var: 0.8552 - val_loss: 137.4548 - val_I_var: 0.7964
Epoch 121/1000
1/1 [==============================] - 0s 49ms/step - loss: 550.6329 - I_var: 0.8579 - val_loss: 137.1076 - val_I_var: 0.7990
Epoch 122/1000
1/1 [==============================] - 0s 43ms/step - loss: 549.0803 - I_var: 0.8606 - val_loss: 136.7635 - val_I_var: 0.8016
Epoch 123/1000
1/1 [========

1/1 [==============================] - 0s 43ms/step - loss: 500.1007 - I_var: 0.9457 - val_loss: 127.3226 - val_I_var: 0.8718
Epoch 175/1000
1/1 [==============================] - 0s 40ms/step - loss: 499.7279 - I_var: 0.9464 - val_loss: 127.2874 - val_I_var: 0.8721
Epoch 176/1000
1/1 [==============================] - 0s 54ms/step - loss: 499.3660 - I_var: 0.9470 - val_loss: 127.2551 - val_I_var: 0.8723
Epoch 177/1000
1/1 [==============================] - 0s 48ms/step - loss: 499.0139 - I_var: 0.9476 - val_loss: 127.2254 - val_I_var: 0.8725
Epoch 178/1000
1/1 [==============================] - 0s 54ms/step - loss: 498.6711 - I_var: 0.9482 - val_loss: 127.1982 - val_I_var: 0.8727
Epoch 179/1000
1/1 [==============================] - 0s 56ms/step - loss: 498.3368 - I_var: 0.9488 - val_loss: 127.1730 - val_I_var: 0.8729
Epoch 180/1000
1/1 [==============================] - 0s 42ms/step - loss: 498.0102 - I_var: 0.9494 - val_loss: 127.1500 - val_I_var: 0.8731
Epoch 181/1000
1/1 [========

1/1 [==============================] - 0s 38ms/step - loss: 483.7813 - I_var: 0.9740 - val_loss: 126.8936 - val_I_var: 0.8747
Epoch 233/1000
1/1 [==============================] - 0s 44ms/step - loss: 483.5154 - I_var: 0.9745 - val_loss: 126.8980 - val_I_var: 0.8746
Epoch 234/1000
1/1 [==============================] - 0s 45ms/step - loss: 483.2501 - I_var: 0.9750 - val_loss: 126.9029 - val_I_var: 0.8746
Epoch 235/1000
1/1 [==============================] - 0s 44ms/step - loss: 482.9854 - I_var: 0.9754 - val_loss: 126.9083 - val_I_var: 0.8745
Epoch 236/1000
1/1 [==============================] - 0s 52ms/step - loss: 482.7215 - I_var: 0.9759 - val_loss: 126.9141 - val_I_var: 0.8745
Epoch 237/1000
1/1 [==============================] - 0s 41ms/step - loss: 482.4584 - I_var: 0.9763 - val_loss: 126.9205 - val_I_var: 0.8744
Epoch 238/1000
1/1 [==============================] - 0s 44ms/step - loss: 482.1960 - I_var: 0.9768 - val_loss: 126.9273 - val_I_var: 0.8744
Epoch 239/1000
1/1 [========

1/1 [==============================] - 0s 41ms/step - loss: 471.4220 - I_var: 0.9955 - val_loss: 128.0975 - val_I_var: 0.8656
Epoch 291/1000
1/1 [==============================] - 0s 43ms/step - loss: 471.2910 - I_var: 0.9957 - val_loss: 128.1325 - val_I_var: 0.8653
Epoch 292/1000
1/1 [==============================] - 0s 56ms/step - loss: 471.1629 - I_var: 0.9959 - val_loss: 128.1678 - val_I_var: 0.8650
Epoch 293/1000
1/1 [==============================] - 0s 41ms/step - loss: 471.0380 - I_var: 0.9961 - val_loss: 128.2031 - val_I_var: 0.8648
Epoch 294/1000
1/1 [==============================] - 0s 56ms/step - loss: 470.9159 - I_var: 0.9964 - val_loss: 128.2387 - val_I_var: 0.8645
Epoch 295/1000
1/1 [==============================] - 0s 52ms/step - loss: 470.7968 - I_var: 0.9966 - val_loss: 128.2744 - val_I_var: 0.8642
Epoch 296/1000
1/1 [==============================] - 0s 52ms/step - loss: 470.6806 - I_var: 0.9968 - val_loss: 128.3102 - val_I_var: 0.8640
Epoch 297/1000
1/1 [========

...

1/1 [==============================] - 0s 44ms/step - loss: 467.0147 - I_var: 1.0032 - val_loss: 131.3612 - val_I_var: 0.8414
Epoch 755/1000
1/1 [==============================] - 0s 48ms/step - loss: 467.0147 - I_var: 1.0032 - val_loss: 131.3612 - val_I_var: 0.8414
Epoch 756/1000
1/1 [==============================] - 0s 52ms/step - loss: 467.0146 - I_var: 1.0032 - val_loss: 131.3612 - val_I_var: 0.8414
Epoch 757/1000
1/1 [==============================] - 0s 41ms/step - loss: 467.0147 - I_var: 1.0032 - val_loss: 131.3611 - val_I_var: 0.8414
Epoch 758/1000
1/1 [==============================] - 0s 39ms/step - loss: 467.0146 - I_var: 1.0032 - val_loss: 131.3611 - val_I_var: 0.8414
Epoch 759/1000
1/1 [==============================] - 0s 41ms/step - loss: 467.0146 - I_var: 1.0032 - val_loss: 131.3611 - val_I_var: 0.8414
Epoch 760/1000
1/1 [==============================] - 0s 49ms/step - loss: 467.0146 - I_var: 1.0032 - val_loss: 131.3611 - val_I_var: 0.8414
Epoch 761/1000
1/1 [========

1/1 [==============================] - 0s 47ms/step - loss: 467.0137 - I_var: 1.0032 - val_loss: 131.3602 - val_I_var: 0.8414
Epoch 813/1000
1/1 [==============================] - 0s 53ms/step - loss: 467.0137 - I_var: 1.0032 - val_loss: 131.3602 - val_I_var: 0.8414
Epoch 814/1000
1/1 [==============================] - 0s 51ms/step - loss: 467.0136 - I_var: 1.0032 - val_loss: 131.3602 - val_I_var: 0.8414
Epoch 815/1000
1/1 [==============================] - 0s 56ms/step - loss: 467.0136 - I_var: 1.0032 - val_loss: 131.3602 - val_I_var: 0.8414
Epoch 816/1000
1/1 [==============================] - 0s 57ms/step - loss: 467.0136 - I_var: 1.0032 - val_loss: 131.3601 - val_I_var: 0.8414
Epoch 817/1000
1/1 [==============================] - 0s 58ms/step - loss: 467.0136 - I_var: 1.0032 - val_loss: 131.3601 - val_I_var: 0.8414
Epoch 818/1000
1/1 [==============================] - 0s 45ms/step - loss: 467.0136 - I_var: 1.0032 - val_loss: 131.3601 - val_I_var: 0.8414
Epoch 819/1000
1/1 [========

1/1 [==============================] - 0s 52ms/step - loss: 467.0127 - I_var: 1.0032 - val_loss: 131.3592 - val_I_var: 0.8414
Epoch 871/1000
1/1 [==============================] - 0s 46ms/step - loss: 467.0127 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 872/1000
1/1 [==============================] - 0s 57ms/step - loss: 467.0126 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 873/1000
1/1 [==============================] - 0s 43ms/step - loss: 467.0126 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 874/1000
1/1 [==============================] - 0s 40ms/step - loss: 467.0126 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 875/1000
1/1 [==============================] - 0s 55ms/step - loss: 467.0125 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 876/1000
1/1 [==============================] - 0s 48ms/step - loss: 467.0126 - I_var: 1.0032 - val_loss: 131.3591 - val_I_var: 0.8414
Epoch 877/1000
1/1 [========

1/1 [==============================] - 0s 42ms/step - loss: 467.0116 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 929/1000
1/1 [==============================] - 0s 39ms/step - loss: 467.0116 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 930/1000
1/1 [==============================] - 0s 50ms/step - loss: 467.0116 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 931/1000
1/1 [==============================] - 0s 56ms/step - loss: 467.0115 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 932/1000
1/1 [==============================] - 0s 47ms/step - loss: 467.0115 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 933/1000
1/1 [==============================] - 0s 45ms/step - loss: 467.0115 - I_var: 1.0032 - val_loss: 131.3580 - val_I_var: 0.8414
Epoch 934/1000
1/1 [==============================] - 0s 38ms/step - loss: 467.0115 - I_var: 1.0032 - val_loss: 131.3579 - val_I_var: 0.8414
Epoch 935/1000
1/1 [========

1/1 [==============================] - 0s 46ms/step - loss: 467.0105 - I_var: 1.0032 - val_loss: 131.3569 - val_I_var: 0.8414
Epoch 987/1000
1/1 [==============================] - 0s 42ms/step - loss: 467.0105 - I_var: 1.0032 - val_loss: 131.3569 - val_I_var: 0.8414
Epoch 988/1000
1/1 [==============================] - 0s 41ms/step - loss: 467.0104 - I_var: 1.0032 - val_loss: 131.3569 - val_I_var: 0.8414
Epoch 989/1000
1/1 [==============================] - 0s 56ms/step - loss: 467.0103 - I_var: 1.0032 - val_loss: 131.3569 - val_I_var: 0.8414
Epoch 990/1000
1/1 [==============================] - 0s 46ms/step - loss: 467.0104 - I_var: 1.0032 - val_loss: 131.3568 - val_I_var: 0.8414
Epoch 991/1000
1/1 [==============================] - 0s 43ms/step - loss: 467.0104 - I_var: 1.0032 - val_loss: 131.3568 - val_I_var: 0.8414
Epoch 992/1000
1/1 [==============================] - 0s 44ms/step - loss: 467.0104 - I_var: 1.0032 - val_loss: 131.3568 - val_I_var: 0.8414
Epoch 993/1000
1/1 [========

## plot global epistasis model

In [ ]:
# load fitted global epistasis model
model = mavenn.load('output/1b_repl1')

# extract local fields h_i
theta = model.get_theta()
h = theta['theta_lc'][:,1] - theta['theta_lc'][:,0]

# predict latent phenotype values (phi) & fitness g(phi) for all sequences
phis = model.x_to_phi(seqs_str)
yhats = model.x_to_yhat(seqs_str)
data['F1_model'] = yhats

# set phi lims and create grid in phi space, compute fitness for each phi gridpoint
phi_lim = [min(phis)-.5, max(phis)+.5]
phi_grid = np.linspace(phi_lim[0], phi_lim[1], 1000)
yhat_grid = model.phi_to_yhat(phi_grid)

In [ ]:
# save fitness dataframe to externaöl file
data.to_csv('output/1c_fitness_global.csv')

In [ ]:
# create figure
fig, ax = plt.subplots(figsize=(1., 3.2))

# color map
hmin, hmax = np.nanmin(h), np.nanmax(h)
cmap = plottoo.shiftedColorMap(mpl.cm.bwr_r, midpoint=abs(hmin)/(hmax+abs(hmin)))

# plot local fields
im = ax.imshow(h[:,np.newaxis], cmap=cmap)

# layout
ax.plot([-.5,.5], [(L-1)/2, (L-1)/2], c='k', linestyle='--')
labels = [r'\texttt{%s}'%st[1:-1] for st in muts]
ax.set_xticks([])
ax.set_yticks(range(L))
ax.set_yticklabels(labels)
ax.tick_params(labelsize=15)

# colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.1, 0.7])
cb = fig.colorbar(im, cax=cbar_ax)
cb.ax.tick_params(labelsize=15)
cb.set_ticks([-.5,0.,.5,1.])

# save plot
plottoo.set_size(.625,2)
plt.savefig('output/1b_1.jpg', bbox_inches='tight', pad_inches=0.02, dpi=300)#_singlemode
plt.savefig('output/1b_1.pdf', bbox_inches='tight', pad_inches=0.02)#_singlemode
plt.show()

In [ ]:
# sequence site of interest
i = 6

# create figure
fig, ax = plt.subplots(figsize=(3.2, 2.8))

# colors according to state of sequence site of interest
cs = ['C0' if s[i]==0 else 'C1' for s in data.seq]

# scatter empirical vs. model fitness
ax.scatter(phis, data.F1_emp, c=cs, s=5, alpha=.5, zorder=5)
ax.plot(phi_grid, yhat_grid, c='k', lw=2, zorder=5)

# layout
ax.set_xlim(phi_lim)
ax.tick_params(labelsize=15)
ax.set_xlabel('latent phenotype $\phi(\mathbf{s})$', fontsize=15)
ax.set_ylabel('$F_\mathrm{emp}(\mathbf{s})$', fontsize=15)
ax.grid(zorder=5)

# legend
handles = [mpatches.Patch(color=c) for c in ['C0', 'C1', 'k']]
ax.legend(handles=handles, labels=[r'wild-type', r'mutated', r'$g(\phi)$'],
          loc='lower right', fontsize=12)

# save plot
plt.savefig('output/1b_2.jpg', bbox_inches='tight', pad_inches=0.02, dpi=300)
plt.savefig('output/1b_2.pdf', bbox_inches='tight', pad_inches=0.02)
plt.show()

In [ ]:
# create landscape object using global fitness model data
lsmodel = lstoo.EmpLS(L=L, q=q, seqs=data.seq, fs=data.F1_model, default=np.nan)

# compute epistatic effect matrix (gamma_ij)
fitness = lambda seq: lsmodel.fitness(seq)
gammaijs = lstoo.gammaij(L, seqs, fitness)

In [ ]:
# create figure
fig, ax = plt.subplots(figsize=(2.8, 2.8), constrained_layout=True)

# plot gamma_ij matrix
im = ax.imshow(gammaijs, cmap=mpl.cm.bwr_r, vmin=-1., vmax=1.)

# layout
ax.plot([-1., L+1], [(L-1)/2, (L-1)/2], c='k', linestyle='--')
ax.plot([(L-1)/2, (L-1)/2], [-1., L+1], c='k', linestyle='--')
labels = [r'\texttt{%s}'%st[1:-1] for st in muts]
ax.set_xticks(range(L))
ax.set_xticklabels(labels, rotation='vertical')#range(1,L+1))
ax.set_yticks(range(L))
ax.set_yticklabels(labels)
ax.tick_params(labelsize=15)
ax.set_xlabel(r'$j$', fontsize=15)
ax.set_ylabel(r'$i$', fontsize=15)
ax.set_xlim([-.5, L-.5])
ax.set_ylim([L-.5, -.5])

# colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=.05)
cb = plt.colorbar(im, cax=cax)
cb.ax.tick_params(labelsize=15)
cb.set_ticks([-1,0,1])

# save plot
plottoo.set_size(2,2)
plt.savefig('output/1b_3.jpg', bbox_inches='tight', pad_inches=0.02, dpi=300)
plt.savefig('output/1b_3.pdf', bbox_inches='tight', pad_inches=0.02)
plt.show()